In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import col, size
import pyspark.sql.functions as fn

In [2]:
# Start spark in local mode using 100gb of memory
# local mode only runs on a single node, but it will utilize all cores (We have 48!)
conf = SparkConf().setAppName("test").set('spark.driver.memory','64g')
#.setMaster("yarn") # this is used when we run on hadoop, ignore for now

sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

print("Spark Version: ", sc.version)
print("defaultParallelism: ", sc.defaultParallelism)
print("Spark WebURLL ", sc.uiWebUrl) # you can view running jobs here, but I am only able to connect to it via VNC rn, maybe SSH tunneling will fix this? idk

Spark Version:  2.4.4
defaultParallelism:  48
Spark WebURLL  http://c251-132.wrangler.tacc.utexas.edu:4040


In [3]:
sc._conf.getAll() # See all the current Spark configuration settings

[('spark.app.id', 'local-1572982312923'),
 ('spark.driver.host', 'c251-132.wrangler.tacc.utexas.edu'),
 ('spark.driver.port', '36358'),
 ('spark.driver.memory', '64g'),
 ('spark.app.name', 'test'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.local.dir', '/data/06271/cju256/temp'),
 ('spark.ui.showConsoleProgress', 'true')]

In [4]:
import json
from pyspark.sql.types import StructType

# load data from the json file (we can also do the csv when we have it)

schema_json = sqlContext.read.text("/data/06271/cju256/flat.schema").first()[0]
schema = StructType.fromJson(json.loads(schema_json))

flat_df = sqlContext.read.json('/data/06271/cju256/ut_venmo_2018_flat.json', schema = schema)

In [5]:
# get the schema of the json
flat_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- actor_about: string (nullable = true)
 |-- actor_cancelled: boolean (nullable = true)
 |-- actor_date_created: string (nullable = true)
 |-- actor_email: string (nullable = true)
 |-- actor_external_id: string (nullable = true)
 |-- actor_firstname: string (nullable = true)
 |-- actor_friends: string (nullable = true)
 |-- actor_id: long (nullable = true)
 |-- actor_is_business: boolean (nullable = true)
 |-- actor_lastname: string (nullable = true)
 |-- actor_name: string (nullable = true)
 |-- actor_num_friends: long (nullable = true)
 |-- actor_phone: string (nullable = true)
 |-- actor_picture: string (nullable = true)
 |-- actor_username: string (nullable = true)
 |-- comments_count: long (nullable = true)
 |-- created_time: string (nullable = true)
 |-- likes_count: long (nullable = true)
 |-- mentions_count: long (nullable = true)
 |-- message: string (nullable = true)
 |-- payment_id: long (nullable = true)
 |-- permalink: string (nu

In [7]:
from pyspark.sql.types import StringType, IntegerType, LongType, TimestampType

flat_df = flat_df.withColumn('epoch_time', col('unix_time').cast(TimestampType()))

In [10]:
from pyspark.sql import functions as F

flat_df.select('*', F.date_format('epoch_time', "yyyy-MM").alias('year_month')) \
            .filter(col('year_month') == F.lit('2018-05')).show()

+---+-----------+---------------+------------------+-----------+-----------------+---------------+-------------+--------+-----------------+--------------+----------+-----------------+-----------+-------------+--------------+--------------+------------+-----------+--------------+-------+----------+---------+------------+----------------+-------------------+------------+------------------+----------------+--------------+---------+------------------+---------------+-----------+------------------+------------+--------------+-----------+---------------+----+---------+------------+----------+----------+
|_id|actor_about|actor_cancelled|actor_date_created|actor_email|actor_external_id|actor_firstname|actor_friends|actor_id|actor_is_business|actor_lastname|actor_name|actor_num_friends|actor_phone|actor_picture|actor_username|comments_count|created_time|likes_count|mentions_count|message|payment_id|permalink|target_about|target_cancelled|target_date_created|target_email|target_external_id|targe

In [11]:
year_month = flat_df.select(F.date_format('epoch_time','yyyy-MM').alias('month')).groupby('month').count().persist()
year_month.orderBy('month').show(200)

+-------+--------+
|  month|   count|
+-------+--------+
|2012-03|      15|
|2012-04|    3280|
|2012-05|    4433|
|2012-06|    4756|
|2012-07|    5571|
|2012-08|    7338|
|2012-09|    9769|
|2012-10|   11965|
|2012-11|   13677|
|2012-12|   16453|
|2013-01|   21337|
|2013-02|   25457|
|2013-03|   33211|
|2013-04|   40643|
|2013-05|   48467|
|2013-06|   50822|
|2013-07|   64736|
|2013-08|   88611|
|2013-09|  120736|
|2013-10|  147528|
|2013-11|  173434|
|2013-12|  192545|
|2014-01|  256247|
|2014-02|  301696|
|2014-03|  410951|
|2014-04|  464851|
|2014-05|  530547|
|2014-06|  524249|
|2014-07|  589398|
|2014-08|  733758|
|2014-09|  901403|
|2014-10| 1049696|
|2014-11| 1120404|
|2014-12| 1156727|
|2015-01| 1412554|
|2015-02| 1568348|
|2015-03| 1962782|
|2015-04| 2128499|
|2015-05| 2312411|
|2015-06| 2040899|
|2015-07| 2245847|
|2015-08| 2642943|
|2015-09| 3161513|
|2015-10| 3567102|
|2015-11| 3528311|
|2015-12| 3602472|
|2016-01| 4322864|
|2016-02| 4691546|
|2016-03| 5384113|
|2016-04| 60

In [6]:
# This registers the dataframe as a table to make SQL queries on
flat_df.registerTempTable('all_table')

# we can treat the data as SQL and run queries!
sqlContext.sql("SELECT message FROM all_table").show()

+--------------------+
|             message|
+--------------------+
|             for tea|
|    for contribution|
|         for Beijing|
|for Phillies game...|
|    for symbioticket|
|for Ameren: month #1|
|for special skate...|
|        for dog food|
|        Weekend food|
|  for CABBB BETCHHHH|
|          for dinner|
|for Sounders Tickets|
|       for remainder|
|              for ,,|
|            👀
👃
👅|
|      for watermelon|
|for LA Halloween ...|
|       for mini food|
|                Grim|
|             Tickets|
+--------------------+
only showing top 20 rows



In [12]:
flat_df.count()

342281006

In [13]:
flat_df.select("payment_id", "message", "actor_id", "target_id").show()

+----------+--------------------+--------+---------+
|payment_id|             message|actor_id|target_id|
+----------+--------------------+--------+---------+
|    482374|             for tea|   59741|    33632|
|    486539|    for contribution|  111497|   110891|
|    486841|         for Beijing|   34421|    22792|
|    462308|for Phillies game...|   85649|   107484|
|    486850|    for symbioticket|   26924|    34907|
|    550450|for Ameren: month #1|  115128|   115131|
|    519086|for special skate...|   74104|    65588|
|    559299|        for dog food|   71313|    60063|
|    569537|        Weekend food|   29674|    22222|
|    610826|  for CABBB BETCHHHH|  116523|   125011|
|    612105|          for dinner|   73816|    27407|
|    581179|for Sounders Tickets|   65739|    53702|
|    571284|       for remainder|   28429|    43575|
|    600315|              for ,,|  130287|   130290|
|    615779|            👀
👃
👅|  116185|   128330|
|    689703|      for watermelon|  129335|   1272

In [14]:
flat_df.agg({"payment_id": "max", "actor_id": "max", "target_id": "max"}).show()

+---------------+--------------+-------------+
|max(payment_id)|max(target_id)|max(actor_id)|
+---------------+--------------+-------------+
|     1160228993|      41493705|     41493680|
+---------------+--------------+-------------+



In [15]:
flat_df.select("payment_id", "actor_num_friends").filter("actor_num_friends > 0").show()

+----------+-----------------+
|payment_id|actor_num_friends|
+----------+-----------------+
+----------+-----------------+



In [ ]:
user_trans = flat_df.select("payment_id", "actor_id").groupby("actor_id").count()

In [ ]:
user_trans.show()

+--------+-----+
|actor_id|count|
+--------+-----+
|  116185|   31|
|  118628|  507|
|  502191|   53|
|  242209|  181|
|  225635|  313|
|  152892|  118|
|  573876|   34|
|  744979|   48|
| 1109864|   48|
| 1136272|  129|
|  215619|  131|
| 1214786|   71|
| 1327889|   17|
|  437239|  173|
|  534151|   32|
|  536821|   59|
|  475022|  222|
|  657366|  186|
| 1770597|  102|
|  586616|  237|
+--------+-----+
only showing top 20 rows



In [ ]:
user_trans.filter("count < 10").count()

12074276

In [ ]:
user_trans.filter("count > 1000").count()

141

In [ ]:
user_trans.printSchema()

root
 |-- actor_id: long (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
flat_df.groupby("actor_id").agg({"unix_time": "max", "unix_time": "min"}).show()

+--------+--------------+
|actor_id|min(unix_time)|
+--------+--------------+
|  116185|    1343703003|
|  118628|    1338644689|
|  502191|    1378346076|
|  242209|    1364933796|
|  225635|    1365375531|
|  152892|    1350593125|
|  573876|    1380405178|
|  744979|    1386389879|
| 1109864|    1395005831|
| 1136272|    1395100090|
|  215619|    1361037737|
| 1214786|    1395510473|
| 1327889|    1399590960|
|  437239|    1377893394|
|  534151|    1381653919|
|  536821|    1390622884|
|  475022|    1377566017|
|  657366|    1383489906|
| 1770597|    1404226464|
|  586616|    1380857586|
+--------+--------------+
only showing top 20 rows



In [ ]:
import pyspark.sql.functions as fn
 
times = flat_df \
    .groupby("actor_id") \
    .agg(fn.max("unix_time").alias("newest_time"), fn.min("unix_time").alias("oldest_time")) \
    .withColumn('lifetime', col('newest_time')-col('oldest_time'))

times.show()

+--------+-----------+-----------+---------+
|actor_id|newest_time|oldest_time| lifetime|
+--------+-----------+-----------+---------+
|  116185| 1418260027| 1343703003| 74557024|
|  118628| 1533248917| 1338644689|194604228|
|  502191| 1523150056| 1378346076|144803980|
|  242209| 1520524433| 1364933796|155590637|
|  225635| 1532999876| 1365375531|167624345|
|  152892| 1533236117| 1350593125|182642992|
|  573876| 1402376090| 1380405178| 21970912|
|  744979| 1478725077| 1386389879| 92335198|
| 1109864| 1514392872| 1395005831|119387041|
| 1136272| 1524339991| 1395100090|129239901|
|  215619| 1524068266| 1361037737|163030529|
| 1214786| 1430752726| 1395510473| 35242253|
| 1327889| 1467322535| 1399590960| 67731575|
|  437239| 1502221230| 1377893394|124327836|
|  534151| 1522565040| 1381653919|140911121|
|  536821| 1524165607| 1390622884|133542723|
|  475022| 1524416480| 1377566017|146850463|
|  657366| 1523830853| 1383489906|140340947|
| 1770597| 1523640332| 1404226464|119413868|
|  586616|

In [ ]:
actors = flat_df.select("actor_id").distinct()
targets = flat_df.select("target_id").distinct()

actors_and_targets = actors.join(targets, actors.actor_id == targets.target_id)

actors_and_targets.show()

+--------+---------+
|actor_id|target_id|
+--------+---------+
|      29|       29|
|    2453|     2453|
|    3506|     3506|
|   21209|    21209|
|   29824|    29824|
|   30928|    30928|
|   32098|    32098|
|   35253|    35253|
|   37261|    37261|
|   37884|    37884|
|   38108|    38108|
|   38510|    38510|
|   38543|    38543|
|   39094|    39094|
|   39627|    39627|
|   40132|    40132|
|   41424|    41424|
|   46424|    46424|
|   47928|    47928|
|   49048|    49048|
+--------+---------+
only showing top 20 rows



In [ ]:
# Inner Join count
actors_and_targets.count()

14494160

In [ ]:
# Left Join count
actors.join(targets, actors.actor_id == targets.target_id, how='left').count()

In [ ]:
# Right Join count
actors.join(targets, actors.actor_id == targets.target_id, how='right').count()

In [ ]:
# Left Exclusive Join Count
# Actors that are not ever targets
actors.join(targets, actors.actor_id == targets.target_id, how='left') \
    .filter("target_id is null") \
    .count()

In [ ]:
# Right Exclusive Join
# Targets that are not ever actors
actors.join(targets, actors.actor_id == targets.target_id, how='right') \
    .filter("actor_id is null") \
    .count()

In [ ]:
# Count of all 
actors_plus_targets = actors.union(targets)

actors_plus_targets.distinct().count()

In [ ]:
times_target = flat_df \
    .groupby("target_id") \
    .agg(fn.max("unix_time").alias("newest_time"), fn.min("unix_time").alias("oldest_time")) \
    .withColumn('lifetime', col('newest_time')-col('oldest_time'))

times_target.show()

In [ ]:
actors = flat_df.select("actor_id", col("actor_date_created").alias("date_created"))

actors.show()

In [ ]:
actors.filter("actor_id == 59741").show()

In [ ]:
import pyspark.sql.functions as fn

all_times = flat_df \
    .agg(fn.max("unix_time").alias("newest_time"), fn.min("unix_time").alias("oldest_time"))

all_times.show()

First Day: Sunday, March 25, 2012 1:01:35 AM GMT-05:00 DST

Last Day: Thursday, August 16, 2018 1:05:47 AM GMT-05:00 DST

In [7]:
ten_k_path = '/data/06271/cju256/ten_k_flat.json'
tenk_df = sqlContext.read.json(ten_k_path)

In [6]:
one_mil = flat_df.limit(1000000)
one_mil.coalesce(1).write.format('json').save('/data/06271/cju256/one_mil.json')